## Import Necessary Packages

In [1]:
import os
import glob
import numpy as np
import xarray as xr
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

## User-Defined Fields

In [2]:
author = # 'your name'
email  = # 'your email'
path   = # '/directory/where/large/or/small/domain/files/are/located/'
simset = 'rcemip-large' # or 'rcemip-small'
dirout = 'FMSE'
cases  = ['nz_32','nz_64','nz_128','nz_256']
today  = datetime.today().strftime('%Y-%m-%d')

In [3]:
path = path+simset
if simset == 'rcemip-small':
    outstr = '_99km_300K_'
    nxny   = '_128_'
else:
    outstr = '_1536km_300K_'
    nxny   = '_512_'
    cases = cases[:2]

## Calculate and Save Frozen Moist Static Energy

In SAM's simple single moment microphysics scheme, there is a simple temperature-dependent partitioning between liquid and ice (Khairoutdinov & Randall 2003). The partition functions $\omega_m$ depend on temperature alone, where $T_{0n} = 285 T_{0p} = 283.16 T_{0g} = 283.16 T_{00n} = 253.16 T_{00p} = 268.16 T_{00g} = 223.16$: 

$$\omega_m = \textrm{max}[0,\textrm{min}(1,\frac{T-T_{00m}}{T_{0m}-T_{00m}})]$$


Using the above equation, calculate the non-precipitating (cloud) ice mixing ratio as $q_i = (1-\omega_n) q_n$, and precipitating ice mixing ratio as $q_{pi} = (1-\omega_p) q_p$. Calculate frozen moist static energy $h_f$ and its mass-weighted vertical integral $\langle h_f \rangle$ following:
$$h_f = c_pT + gz + L_vq_v - L_f(q_i+q_{pi})$$
$$\langle h_f \rangle = \frac{1}{g} \int_{p_{bottom}}^{p_{top}} (h_f) dp$$

Store $h_f$ and $\langle h_f \rangle$ together in Xarray.Datasets by case, and save as netCDF files to ```dirout```.

In [4]:
T_0n  = 285
T_0p  = 283.16
T_00n = 253.16
T_00p = 268.16
cp = 1006
g  = 9.81
Lv = 2.5e6
Lf = 3.34e5

In [5]:
for i,case in enumerate(cases):
    ## Check to See if Directory Already Exists
    try: os.mkdir(path+case+'/OUT_3D/'+dirout)
    except: print('Error: directory already exists')
    print('Working on '+case+'...')
    ## List Files
    pfile   = glob.glob(path+case+'/OUT_3D/p/*.nc')[0]
    tafiles = np.sort(glob.glob(path+case+'/OUT_3D/TABS/*.nc'))
    qnfiles = np.sort(glob.glob(path+case+'/OUT_3D/QN/*.nc'))
    qpfiles = np.sort(glob.glob(path+case+'/OUT_3D/QP/*.nc'))
    qvfiles = np.sort(glob.glob(path+case+'/OUT_3D/QV/*.nc'))
    ppfiles = np.sort(glob.glob(path+case+'/OUT_3D/PP/*.nc'))
    for j in range(len(tafiles)):
        ## Check to See if File Exists
        timestr = tafiles[j][-13:-3]
        print(str(i)+'/'+str(len(tafiles)))
        filename = (path+case+'/OUT_3D/'+dirout+'/RCE_'+simset+outstr+case+nxny+dirout+'_'+timestr+'.nc')
        if os.path.exists(filename): 
                print('Error: file already exists and will be skipped')
                continue
        ## Define Variables
        p_background = xr.open_dataset(pfile).p*100 # from hPa to Pa
        p  = xr.open_dataset(ppfiles[j]).PP + p_background
        ta = xr.open_dataset(tafiles[j]).TABS
        qn = xr.open_dataset(qnfiles[j]).QN
        qp = xr.open_dataset(qpfiles[j]).QP
        qv = xr.open_dataset(qvfiles[j]).QV
        ## Calculate Precipitating Ice Mixing Ratio
        omega_n = xr.ufuncs.maximum(0,xr.ufuncs.minimum(1,(ta-T_00n)/(T_0n-T_00n)))
        qi = (1-omega_n)*qn/1000 # from g/kg to kg/kg 
        del omega_n
        del qn
        ## Calculate Precipitating Ice Mixing Ratio
        omega_p = xr.ufuncs.maximum(0,xr.ufuncs.minimum(1,(ta-T_00p)/(T_0p-T_00p)))
        qpi = (1-omega_p)*qp/1000 # from g/kg to kg/kg 
        del omega_p
        del qp
        ## Calculate FMSE
        hf = ((cp*ta)+(g*ta.z)+(Lv*(qv/1000))-(Lf*(qi+qpi))) # from g/kg to kg/kg
        hf.attrs['long_name'] = 'Frozen moist static energy'
        hf.attrs['units'] = '$J kg^{-1}$'
        hf.attrs['description'] = '$c_pT + gz + L_vq_v - L_fq_i$'
        del ta
        del qv
        del qi
        del qpi
        ## Calculate Column FMSE
        dp = xr.ufuncs.fabs(p.diff(dim='z'))
        hf_vint = (1/g)*(h_i[:,:-1,:,:]*dp).sum(dim='z')
        hf_vint.attrs['long_name'] = 'Mass-weighted vertically-integrated frozen moist static energy'
        hf_vint.attrs['units'] = 'J m$^{-2}$'
        hf_vint.attrs['description'] = 'Mass weighting calculated as sum($h_fdp$)/$g$'
        ## Create Dataset and Save as a NetCDF File
        print('Creating and saving dataset to: '+filename)
        ds = xr.Dataset(data_vars=dict(fmse=hf,fmse_vint=hf_vint),
                        attrs=dict(history='Calculated on '+today+' by '+author+': '+email))
        ds.to_netcdf(filename)
        del p
        del hf
        del hf_vint
        del ds